In [139]:
import pandas as pd
import numpy as np
from random import sample
from random import seed
from scipy.stats import ttest_ind
from Ames_Functions import *
from scipy.stats import chi2_contingency
pd.set_option("display.max_columns", None)

In [137]:
df_features_all = load_state_pkl()['df_house_both_after_feature_engineering'][0]
saleprice_2019 = load_state_pkl()["df_RE_latlong2019"][0].SalePrice
saleprice_2021 = load_state_pkl()["df_RE_latlong2021"][0].SalePrice
df_biz = pd.read_csv("df_EssentialLatLong_v2.csv")
df_house2019 = load_state_pkl()["df_RE_latlong2019"][0]
df_house2021 = load_state_pkl()["df_RE_latlong2021"][0]
df_house_all = df_house2019.append(df_house2021).set_index("SaleID")

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_1983/4067500438.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_house_all = df_house2019.append(df_house2021).set_index("SaleID")


In [29]:
# Migrated this to Ames_Functions.py
# def remove_undeveloped_lots (df):
#     return df.copy().query("YrBuilt <= YrSold_YYYY and YrBuilt != 0 and GLA > 0")

In [4]:
df_house_all#.MA_Zip1.value_counts()
selected_AB_house_dummies = ['ZngCdPr', 'ClassSc_S', 'Inst1TPr', 'ValType', 'Neighborhood', 'ParType', 'BldgType', 'Foundation', 'RoofMatl', 'Ext1', 'Ext2', 'MasVnrType', 'Heating', 'Central Air', 'Cars', 'MoSold_MM', 'SaleCond', 'NmbrBRs']

selected_AB_house_dummies

['ZngCdPr',
 'ClassSc_S',
 'Inst1TPr',
 'ValType',
 'Neighborhood',
 'ParType',
 'BldgType',
 'Foundation',
 'RoofMatl',
 'Ext1',
 'Ext2',
 'MasVnrType',
 'Heating',
 'Central Air',
 'Cars',
 'MoSold_MM',
 'SaleCond',
 'NmbrBRs']

In [6]:
#selected_AB_house_dummies
df_AB_house_dummies = (pd.get_dummies(df_house_all[selected_AB_house_dummies]
                                            .drop(["Cars", "MoSold_MM", "NmbrBRs"], axis = 1)
                                            .join(df_house_all[["Cars", "MoSold_MM", "NmbrBRs"]]
                                                    .apply(pd.Categorical, axis = 0)
                                                    )
                                        )
                        # .join(df_house_all.YrSold_YYYY)
                        # .eval("Year = YrSold_YYYY")
                        # .drop("YrSold_YYYY", axis = 1)
                        )
                        
df_AB_house_dummies

,ZngCdPr_A,ZngCdPr_F-PRD,ZngCdPr_F-VR,ZngCdPr_FS-RL,ZngCdPr_FS-RM,ZngCdPr_GI,ZngCdPr_HOC,ZngCdPr_NC,ZngCdPr_RH,ZngCdPr_RL,ZngCdPr_RM,ZngCdPr_S-SMD,ZngCdPr_UCRM,ClassSc_S_1-1/2 STORY - UNFINISHED ALL AGES,ClassSc_S_1-1/2 STORY CONDO - ALL AGES,ClassSc_S_1-1/2 STORY FINISHED ALL AGES,ClassSc_S_1-1/2 STORY TWNHM - ALL AGES,ClassSc_S_1-LEVEL CONDO 1946 & NEWER,ClassSc_S_1-STORY 1945 & OLDER,ClassSc_S_1-STORY 1946 & NEWER ALL STYLES,ClassSc_S_1-STORY TWNHM - 1946 & NEWER,ClassSc_S_2 FAMILY CONVERSION - ALL STYLES AND AGES,ClassSc_S_2-1/2 STORY ALL AGES,ClassSc_S_2-LEVEL CONDO 1946 & NEWER,ClassSc_S_2-STORY 1945 & OLDER,ClassSc_S_2-STORY 1946 & NEWER,ClassSc_S_2-STORY TWNHM - 1946 & NEWER,ClassSc_S_BED & BREAKFAST,ClassSc_S_DUPLEX - ALL STYLES AND AGES,ClassSc_S_GARAGE UNIT-CONDO,ClassSc_S_MULTILEVEL TWNHM - INCL SPLIT LEV/FOYER,ClassSc_S_SPLIT FOYER,ClassSc_S_SPLIT OR MULTI-LEVEL,ClassSc_S_STORAGE UNIT-CONDO,ClassSc_S_VACANT LAND - IMPROVED OR UNIMPROVED,Inst1TPr_AFFLEST,Inst1TPr_CNDO,Inst1TPr_COD,Inst1TPr_COTAFF,Inst1TPr_CTR,Inst1TPr_D,Inst1TPr_ESMT,Inst1TPr_MSC,Inst1TPr_O2,Inst1TPr_Q,Inst1TPr_SHD,Inst1TPr_SWD,Inst1TPr_WD,Inst1TPr_WRDJSV,ValType_B,ValType_F,ValType_I,ValType_L,ValType_P,Neighborhood_Blmngtn,Neighborhood_Bluestm,Neighborhood_Br'Dale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_DakotaR,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_GrnHill,Neighborhood_HaydnLk,Neighborhood_IDOT&RR,Neighborhood_IOCondo,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_MsCondo,Neighborhood_N Ames,Neighborhood_NRidgHt,Neighborhood_NW Ames,Neighborhood_NoPkVil,Neighborhood_NoRidge,Neighborhood_OldTown,Neighborhood_S&W ISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_Stonebr,Neighborhood_Timber,Neighborhood_Veenker,Neighborhood_WllwCr1,Neighborhood_WllwCr2,ParType_Bld&Lot,ParType_Dwg&Lot,ParType_Vac.Lot,BldgType_1FamDet,BldgType_2FmConv,BldgType_Condo,BldgType_Duplex,BldgType_No Data,BldgType_Twnhs-E,BldgType_Twnhs-I,Foundation_Brk/Til,Foundation_C'Block,Foundation_No Data,Foundation_P'Conc,Foundation_Slab,Foundation_Stone,Foundation_Wood,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_No Data,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Ext1_AsbShng,Ext1_BrkComm,Ext1_BrkFace,Ext1_C'Block,Ext1_CemntBd,Ext1_HdBoard,Ext1_MetalSd,Ext1_No Data,Ext1_Plywood,Ext1_Stucco,Ext1_VinylSd,Ext1_Wd Sdng,Ext1_WdShing,Ext2_AsbShng,Ext2_AsphShn,Ext2_Brk Cmn,Ext2_BrkFace,Ext2_C'Block,Ext2_CmentBd,Ext2_HdBoard,Ext2_ImStucc,Ext2_MetalSd,Ext2_No Data,Ext2_Other,Ext2_Plywood,Ext2_Stone,Ext2_Stucco,Ext2_VinylSd,Ext2_Wd Sdng,Ext2_Wd Shng,MasVnrType_Brk Cmn,MasVnrType_BrkFace,MasVnrType_No Data,MasVnrType_None,MasVnrType_Stone,Heating_GasFWA,Heating_GasHWat,Heating_Geothrm,Heating_Gravity,Heating_HeatPmp,Heating_No Data,Heating_OthElec,Heating_WallHtr,Central Air_No,Central Air_No Data,Central Air_Yes,SaleCond_Abnorml,SaleCond_AdjLand,SaleCond_Alloca,SaleCond_Family,SaleCond_Normal,SaleCond_Partial,Cars_0.0,Cars_0.78,Cars_0.8,Cars_0.85,Cars_1.0,Cars_1.12,Cars_1.16,Cars_2.0,Cars_3.0,Cars_4.0,MoSold_MM_1.0,MoSold_MM_2.0,MoSold_MM_3.0,MoSold_MM_4.0,MoSold_MM_5.0,MoSold_MM_6.0,MoSold_MM_7.0,MoSold_MM_8.0,MoSold_MM_9.0,MoSold_MM_10.0,MoSold_MM_11.0,MoSold_MM_12.0,NmbrBRs_0.0,NmbrBRs_1.0,NmbrBRs_2.0,NmbrBRs_3.0,NmbrBRs_4.0,NmbrBRs_5.0,NmbrBRs_6.0,NmbrBRs_7.0
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
93

In [227]:
#df_AB_house_dummies#.max().rename("maximums").to_frame().query("maximums > 1")
#pd.get_dummies(pd.Categorical(df_house_all[selected_AB_house_dummies].Cars))
df_house_all[selected_AB_house_dummies].drop(["Cars", "MoSold_MM", "NmbrBRs"], axis = 1).join(df_house_all[selected_AB_house_dummies][["Cars", "MoSold_MM", "NmbrBRs"]].apply(pd.Categorical, axis = 0))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1534 entries, 92 to 22888
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ZngCdPr       1534 non-null   object  
 1   ClassSc_S     1534 non-null   object  
 2   Inst1TPr      1534 non-null   object  
 3   ValType       1534 non-null   object  
 4   Neighborhood  1534 non-null   object  
 5   ParType       1534 non-null   object  
 6   BldgType      1534 non-null   object  
 7   Foundation    1534 non-null   object  
 8   RoofMatl      1534 non-null   object  
 9   Ext1          1534 non-null   object  
 10  Ext2          1534 non-null   object  
 11  MasVnrType    1534 non-null   object  
 12  Heating       1534 non-null   object  
 13  Central Air   1534 non-null   object  
 14  SaleCond      1534 non-null   object  
 15  Cars          1534 non-null   category
 16  MoSold_MM     1534 non-null   category
 17  NmbrBRs       1534 non-null   category
dtypes: cat

## Hypothesis testing numeric fields using ttest

In [79]:
#df_house_all[["GLA", "YrSold_YYYY"]]
df_features_all

,Rcrd_Yr,InstOne_Yr,TtlVal_AsrYr,LotArea,YrBuilt,TtlBsmtSF,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,Has_Fireplace,Has_Garage,Has_Bsmt,TotalArea,TotalArea_Gar,AsbShng_Ext,AsphShn_Ext,BrkComm_Ext,BrkFace_Ext,CBlock_Ext,CemntBd_Ext,HdBoard_Ext,ImStucc_Ext,MetalSd_Ext,No_Data_Ext,Other_Ext,Plywood_Ext,Stucco_Ext,VinylSd_Ext,Wd_Sdng_Ext,WdShing_Ext,Two_Types_GrType,Attachd_GrType,Basment_GrType,BuiltIn_GrType,Carport_GrType,Detachd_GrType,No_Data_GrType,None_GrType,One_Story_HsStyle,One_Five_Fin_HsStyle,One_Five_Unf_HsStyle,Two_Story_HsStyle,Two_Five_Fin_HsStyle,Two_Five_Unf_HsStyle,No_Data_HsStyle,S_Foyer_HsStyle,S_Level_HsStyle,Closest_Time,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Gym_closest,Service_High_School_closest,Service_Historic_closest,Service_ISU_closest,Service_Library_closest,Service_Medical_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Closest_5,Convenience_secs,Central_historic_feature,biz_180_secs,biz_480_secs,Year,Stone_Ext
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,2019.0,2019.0,300,8746.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
93,2019.0,2019.0,300,9358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
96,2019.0,2019.0,389500,10015.0,2016.0,1142.0,1.0,0.0,2016.0,3.0,647.0,1,1,1,3385.0,4032.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,243.1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,339.06,487.6,524.0,0,8,2019,0.0
98,2019.0,2019.0,330800,10280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,244.3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,340.26,488.8,525.2,0,8,2019,0.0
100,2021.0,2021.0,399800,10840.0,2020.0,1659.0,0.0,0.0,2020.0,3.0,791.0,0,1,1,3318.0,4109.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,256.8,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,352.76,501.3,537.7,0,8,2019,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,2021.0,2021.0,177600,10776.0,1991.0,950.0,0.0,0.0,2001.0,2.0,747.0,0,1,1,1929.0,2676.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,276.6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,295.28,402.4,391.4,0,15,2021,0.0
22749,2021.0,2021.0,167300,12395.0,1984.0,864.0,0.0,0.0,1984.0,2.0,484.0,0,1,1,1753.0,2237.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,305.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,323.88,431.0,420.0,0,14,2021,0.0
22756,2021.0,2021.0,162900,10463.0,1991.0,962.0,0.0,0.0,1993.0,1.0,384.0,0,1,1,1963.0,2347.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,324.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342.68,449.8,438.8,0,14,2021,0.0


In [98]:
def ttest_2019_2021 (field):
    df_filter = df_features_all.pipe(remove_undeveloped_lots, df_house_all)
    return ttest_ind(df_filter.query('Year == 2019')[field], df_filter.query('Year == 2021')[field])

In [109]:
df_features_all.pipe(remove_undeveloped_lots, df_house_all)

,Rcrd_Yr,InstOne_Yr,TtlVal_AsrYr,LotArea,YrBuilt,TtlBsmtSF,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,Has_Fireplace,Has_Garage,Has_Bsmt,TotalArea,TotalArea_Gar,AsbShng_Ext,AsphShn_Ext,BrkComm_Ext,BrkFace_Ext,CBlock_Ext,CemntBd_Ext,HdBoard_Ext,ImStucc_Ext,MetalSd_Ext,No_Data_Ext,Other_Ext,Plywood_Ext,Stucco_Ext,VinylSd_Ext,Wd_Sdng_Ext,WdShing_Ext,Two_Types_GrType,Attachd_GrType,Basment_GrType,BuiltIn_GrType,Carport_GrType,Detachd_GrType,No_Data_GrType,None_GrType,One_Story_HsStyle,One_Five_Fin_HsStyle,One_Five_Unf_HsStyle,Two_Story_HsStyle,Two_Five_Fin_HsStyle,Two_Five_Unf_HsStyle,No_Data_HsStyle,S_Foyer_HsStyle,S_Level_HsStyle,Closest_Time,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Gym_closest,Service_High_School_closest,Service_Historic_closest,Service_ISU_closest,Service_Library_closest,Service_Medical_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Closest_5,Convenience_secs,Central_historic_feature,biz_180_secs,biz_480_secs,Year,Stone_Ext,GLA,YrSold_YYYY
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96,2019.0,2019.0,389500,10015.0,2016.0,1142.0,1.0,0.0,2016.0,3.0,647.0,1,1,1,3385.0,4032.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,243.1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,339.06,487.6,524.0,0,8,2019,0.0,2243.0,2019.0
112,2020.0,2020.0,403900,9808.0,2019.0,1616.0,1.0,0.0,2019.0,3.0,793.0,1,1,1,3232.0,4025.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,274.2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,369.26,517.4,555.1,0,8,2019,0.0,1616.0,2019.0
114,2020.0,2020.0,350000,9856.0,2019.0,1438.0,1.0,0.0,2019.0,2.0,600.0,1,1,1,2876.0,3476.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,284.8,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,374.56,522.2,565.7,0,8,2019,0.0,1438.0,2019.0
115,2019.0,2019.0,357700,10011.0,2018.0,1474.0,1.0,0.0,2018.0,3.0,642.0,1,1,1,2948.0,3590.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,285.6,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,374.72,522.5,566.5,0,8,2019,0.0,1474.0,2019.0
117,2019.0,2019.0,344600,10172.0,2018.0,1421.0,1.0,0.0,2018.0,3.0,644.0,1,1,1,2842.0,3486.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,286.4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,374.88,522.9,567.3,0,8,2019,0.0,1421.0,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,2021.0,2021.0,177600,10776.0,1991.0,950.0,0.0,0.0,2001.0,2.0,747.0,0,1,1,1929.0,2676.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,276.6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,295.28,402.4,391.4,0,15,2021,0.0,979.0,2021.0
22749,2021.0,2021.0,167300,12395.0,1984.0,864.0,0.0,0.0,1984.0,2.0,484.0,0,1,1,1753.0,2237.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,305.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,323.88,431.0,420.0,0,14,2021,0.0,889.0,2021.0
22756,2021.0,2021.0,162900,10463.0,1991.0,962.0,0.0,0.0,1993.0,1.0,384.0,0,1,1,1963.0,2347.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,324.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342.68,449.8,438.8,0,14,2021,0.0,1001.0,2021.0


In [112]:
sig_fields = (pd.DataFrame([[n, ttest_2019_2021(n)[1]] for n in ['TotalArea_Gar', 'TtlVal_AsrYr', 'YrBuilt', 'Cars', 'Closest_Time', 'Closest_5', 'Convenience_secs', 'Central_historic_feature', 'biz_180_secs', 'biz_480_secs']],
               columns = ["field", "p_value"])
                .query("p_value < 0.05").field.values
)


In [113]:
sig_fields

array(['Closest_5', 'Convenience_secs'], dtype=object)

In [114]:
display(df_features_all.query("Year == 2019")[sig_fields].describe(), df_features_all.query("Year == 2021")[sig_fields].describe())

,Closest_5,Convenience_secs
count,901.000000,901.000000
mean,230.535072,370.665927
std,106.177163,122.704920
min,36.920000,145.000000
25%,137.080000,268.300000
50%,213.460000,373.400000
75%,326.120000,462.700000
max,477.300000,640.200000


,Closest_5,Convenience_secs
count,633.000000,633.000000
mean,216.309131,356.659558
std,99.695220,118.336055
min,20.880000,134.600000
25%,136.700000,250.200000
50%,201.840000,360.600000
75%,297.360000,447.500000
max,456.500000,640.200000


In [9]:
ttest_ind(df_features_all.query("YrBuilt != 0 and Year == 2019").YrBuilt, df_features_all.query("YrBuilt != 0 and Year == 2021").YrBuilt)

Ttest_indResult(statistic=2.2761912991955637, pvalue=0.02297497305100568)

In [120]:
(df_features_all
    .pipe(remove_undeveloped_lots, df_house_all)
    .groupby('Year')
    .describe()[sig_fields]
    .T
    .reset_index()
    .query("level_1 == 'mean'")
    .drop("level_1", axis = 1)
    .set_axis(["Feature", "2019", "2021"], axis = 1)
).to_csv("Visualizations/Ttest_features_results.csv", index = None)

## AB Testing

In [ ]:
df_house
# Has_Fireplace, Has_Garage, Has_Bsmt, Service_Organic_Groceries_closest

In [33]:
df_features_all.groupby("Year").Has_Fireplace.value_counts().rename("Counts")[(2019, 0)]

407

In [14]:
def AB_test (df_features_all, feature):
    total = df_features_all.shape[0]
    group2019 = df_features_all.query("Year == 2019").shape[0]
    group2021 = df_features_all.query("Year == 2021").shape[0]
    if df_features_all.groupby("Year")[feature].value_counts().shape[0] < 4:
        return 1,1,1
    group2019_yes = df_features_all.groupby("Year")[feature].value_counts().rename("Counts")[(2019, 1)]
    group2021_yes = df_features_all.groupby("Year")[feature].value_counts().rename("Counts")[(2021, 1)]
    total_yes = group2019_yes + group2021_yes
    total_no = total - total_yes
    group2019_yes_pc = 100 * group2019_yes / group2019
    group2021_yes_pc = 100 * group2021_yes / group2021
    ab_yes_pc = group2019_yes_pc - group2021_yes_pc
    seed(0)
    bag1 = [1]*total_yes + [0]*total_no
    perms = 100
    perm_res = [0]*perms

    for i in range(perms):
        bag2 = sample(bag1, len(bag1))
        group2019_rs = bag2[:group2019]
        group2021_rs = bag2[group2019:]
        group2019_yes_pc_rs = 100 *sum(group2019_rs)/group2019
        group2021_yes_pc_rs = 100 *sum(group2021_rs)/group2021
        perm_res[i] = group2019_yes_pc_rs - group2021_yes_pc_rs

    s_perm_res = pd.Series(perm_res)
    return (sum(s_perm_res.abs() >= abs(ab_yes_pc))/perms, sum(s_perm_res >= ab_yes_pc)/perms, sum(s_perm_res <= ab_yes_pc)/perms)

In [ ]:
#df_features_all.join(df_AB_house_dummies)
#df_AB_house_dummies.columns.tolist()
df_features_all.filter(regex = "_closest|_GrType|_HsStyle|Has_").columns.tolist()

In [133]:
selected_AB_features = (df_features_all.filter(regex = "_closest|_GrType|_HsStyle|Has_").columns.tolist() +
                        df_AB_house_dummies.columns.tolist()
                        )

df_features_all_dummies = df_features_all.join(df_AB_house_dummies).join(df_house_all[["YrSold_YYYY", "GLA"]]).pipe(remove_undeveloped_lots)

df_AB_final_results = pd.DataFrame([[result[0], result[1][1], result[1][2]] for result in [[f, AB_test(df_features_all_dummies, f)] for f in selected_AB_features] if result[1][1] <= 0.05 or result[1][2] <= 0.05],
                                         columns = ["Name", "pvalue_2019", "pvalue_2021"])

In [134]:
df_AB_Year_Favored = df_AB_final_results.join(df_AB_final_results
                                .apply(lambda x: 2019 if x.pvalue_2019 < x.pvalue_2021 else 2021, axis = 1)
                                .rename("Year_Favored")
                            )      



In [135]:
def print_favored_features (feature):
    year_dict = {2019: [], 2021: []}
    for y in [2019, 2021]: 
        if df_AB_Year_Favored.query("Year_Favored == @y and Name.str.contains(@feature)").shape[0] > 0:
            year_dict[y] = df_AB_Year_Favored[["Name", "Year_Favored"]].query("Name.str.contains(@feature)").pivot(columns = "Year_Favored", values = "Name")[y].dropna().reset_index(drop = True)
    return year_dict

rows_list = []

for f in [ff for ff in selected_AB_house_dummies if df_AB_Year_Favored.query("Name.str.contains(@ff)").shape[0] > 0]:
    results = print_favored_features(f)
    s_f, s_2019, s_2021 = pd.Series([f], name = "Feature"), pd.Series(results[2019], dtype = float, name = 2019), pd.Series(results[2021], dtype = float, name = 2021)
    rows_list.append(pd.concat([s_f, s_2019, s_2021], axis = 1))
    #rows_list.append(pd.DataFrame([None, None, None], index = ["Feature", 2019, 2021]).T)
# with open("Visualizations/Favored_features_all.html", "w") as f:
#     f.write(pd.concat(rows_list)
#                 .fillna(" ")
#                 .replace(r"Neighborhood_|ZngCdPr_|ValType_|ParType_|BldgType_|Foundation_|RoofMatl_|Ext1_|Ext2_|Heating_|Central Air_|Cars_|MoSold_MM_|SaleCond_|NmbrBRs_", "", regex = True)
#                 .style
#                 .hide()
#                 .to_html()
#             )

In [136]:
(pd.concat(rows_list)
    .fillna(" ")
    .replace(r"Neighborhood_|ZngCdPr_|ValType_|ParType_|BldgType_|Foundation_|RoofMatl_|Ext1_|Ext2_|Heating_|Central Air_|Cars_|MoSold_MM_|SaleCond_|NmbrBRs_", "", regex = True)
    #.to_csv("Visualizations/Favored_features_all_ABtest.csv", index = False)
)

,Feature,2019,2021
0,ZngCdPr,RH,
0,ValType,,B
0,Neighborhood,SawyerW,NoRidge
1,,,OldTown
0,BldgType,Twnhs-E,
0,Ext1,,WdShing
0,Ext2,,Wd Shng
0,Heating,,Gravity
0,Central Air,Yes,No
0,Cars,,4.0


In [423]:
s_a, s_b = pd.Series([5, 10 ,15]), pd.Series([], dtype = float, name = 2019)
pd.concat([s_a, s_b], axis = 1)#.reset_index(level = 1, drop = True)

,0,2019
0,5,NaN
1,10,NaN
2,15,NaN


NameError: name 'df_AB_house_dummies' is not defined

In [270]:
df_AB_house_dummies.columns.tolist()[15:20]

['ClassSc_S_1-1/2 STORY FINISHED ALL AGES',
 'ClassSc_S_1-1/2 STORY TWNHM - ALL AGES',
 'ClassSc_S_1-LEVEL CONDO 1946 & NEWER',
 'ClassSc_S_1-STORY 1945 & OLDER',
 'ClassSc_S_1-STORY 1946 & NEWER ALL STYLES']

In [90]:
np.array(df_features_all.filter(regex = "_closest").columns)

array(['Service_Arts_closest', 'Service_Elem_School_closest',
       'Service_Golf_closest', 'Service_Gym_closest',
       'Service_High_School_closest', 'Service_Historic_closest',
       'Service_ISU_closest', 'Service_Library_closest',
       'Service_Medical_closest', 'Service_Organic_Groceries_closest',
       'Service_Park_closest', 'Service_Recreation_closest',
       'Service_Religion_closest', 'Service_Restaurant_closest',
       'Service_Shopping_closest', 'Service_Spa_closest'], dtype=object)

In [75]:
df_features_all

,Rcrd_Yr,InstOne_Yr,TtlVal_AsrYr,LotArea,YrBuilt,TtlBsmtSF,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,Has_Fireplace,Has_Garage,Has_Bsmt,TotalArea,TotalArea_Gar,AsbShng_Ext,AsphShn_Ext,BrkComm_Ext,BrkFace_Ext,CBlock_Ext,CemntBd_Ext,HdBoard_Ext,ImStucc_Ext,MetalSd_Ext,No_Data_Ext,Other_Ext,Plywood_Ext,Stucco_Ext,VinylSd_Ext,Wd_Sdng_Ext,WdShing_Ext,Two_Types_GrType,Attachd_GrType,Basment_GrType,BuiltIn_GrType,Carport_GrType,Detachd_GrType,No_Data_GrType,None_GrType,One_Story_HsStyle,One_Five_Fin_HsStyle,One_Five_Unf_HsStyle,Two_Story_HsStyle,Two_Five_Fin_HsStyle,Two_Five_Unf_HsStyle,No_Data_HsStyle,S_Foyer_HsStyle,S_Level_HsStyle,Closest_Time,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Gym_closest,Service_High_School_closest,Service_Historic_closest,Service_ISU_closest,Service_Library_closest,Service_Medical_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Closest_5,Convenience_secs,Central_historic_feature,biz_180_secs,biz_480_secs,Year,Stone_Ext
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,2019.0,2019.0,300,8746.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
93,2019.0,2019.0,300,9358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
96,2019.0,2019.0,389500,10015.0,2016.0,1142.0,1.0,0.0,2016.0,3.0,647.0,1,1,1,3385.0,4032.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,243.1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,339.06,487.6,524.0,0,8,2019,0.0
98,2019.0,2019.0,330800,10280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,244.3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,340.26,488.8,525.2,0,8,2019,0.0
100,2021.0,2021.0,399800,10840.0,2020.0,1659.0,0.0,0.0,2020.0,3.0,791.0,0,1,1,3318.0,4109.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,256.8,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,352.76,501.3,537.7,0,8,2019,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,2021.0,2021.0,177600,10776.0,1991.0,950.0,0.0,0.0,2001.0,2.0,747.0,0,1,1,1929.0,2676.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,276.6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,295.28,402.4,391.4,0,15,2021,0.0
22749,2021.0,2021.0,167300,12395.0,1984.0,864.0,0.0,0.0,1984.0,2.0,484.0,0,1,1,1753.0,2237.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,305.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,323.88,431.0,420.0,0,14,2021,0.0
22756,2021.0,2021.0,162900,10463.0,1991.0,962.0,0.0,0.0,1993.0,1.0,384.0,0,1,1,1963.0,2347.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,324.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342.68,449.8,438.8,0,14,2021,0.0


In [40]:
# group2021_yes_pc, group2019_yes_pc, ab_yes_pc

(49.13112164296999, 54.827968923418425, 5.696847280448438)

In [46]:
# seed(0)
# bag1 = [1]*total_yes + [0]*total_no
# perms = 100
# perm_res = [0]*perms

# for i in range(perms):
#     bag2 = sample(bag1, len(bag1))
#     group2019_rs = bag2[:group2019]
#     group2021_rs = bag2[group2019:]
#     group2019_yes_pc_rs = 100 *sum(group2019_rs)/group2019
#     group2021_yes_pc_rs = 100 *sum(group2021_rs)/group2021
#     perm_res[i] = group2019_yes_pc_rs - group2021_yes_pc_rs

# s_perm_res = pd.Series(perm_res)

In [61]:
# group2021_yes_pc_rs, group2019_yes_pc_rs, ab_yes_pc_rs
s_perm_res.value_counts().sort_index().to_frame().T

,-6.406608,-5.330745,-4.523848,-4.254883,-3.447986,-3.179020,-2.910054,-2.641089,-2.372123,-2.103157,-1.834192,-1.565226,-1.296260,-1.027295,-0.758329,-0.489363,-0.220398,0.048568,0.317534,0.586499,0.855465,1.124431,1.393396,1.662362,1.931328,2.200294,2.469259,2.738225,3.007191,3.276156,3.545122,4.083053,4.352019,4.620985,5.158916,5.427882,5.965813,6.234779,6.772710
0,1,1,1,4,4,1,4,4,3,3,4,2,4,3,4,3,5,5,3,7,1,2,1,4,2,4,3,5,1,2,1,1,1,1,1,1,1,1,1


In [64]:
#A/B Testing Lecture Notes Method
#pd.pivot_table(s_perm_res.value_counts().reset_index(), values = 0, columns = "index")

index,-6.406608,-5.330745,-4.523848,-4.254883,-3.447986,-3.179020,-2.910054,-2.641089,-2.372123,-2.103157,-1.834192,-1.565226,-1.296260,-1.027295,-0.758329,-0.489363,-0.220398,0.048568,0.317534,0.586499,0.855465,1.124431,1.393396,1.662362,1.931328,2.200294,2.469259,2.738225,3.007191,3.276156,3.545122,4.083053,4.352019,4.620985,5.158916,5.427882,5.965813,6.234779,6.772710
0,1,1,1,4,4,1,4,4,3,3,4,2,4,3,4,3,5,5,3,7,1,2,1,4,2,4,3,5,1,2,1,1,1,1,1,1,1,1,1


In [70]:
sum(s_perm_res.abs() >= abs(ab_yes_pc))/perms
sum(s_perm_res >= ab_yes_pc)/perms

0.03

## $Chi^2$ Tests

In [143]:
#This test is specific to the "Number of Houses Sold per Month" plot in Visualizations notebook and slide 8 in presentation
chi2_contingency(df_house_all
    .apply(lambda x: x if x.name != "YrSold_YYYY" else x.astype(int))
    .groupby(["YrSold_YYYY"]).MoSold_MM
    .value_counts()
    .unstack(level = 0)
    .fillna(0)
    .reset_index()
    .query("MoSold_MM < 9.0")
    .filter([2019, 2021])
)[1]

0.04588987198520902

In [148]:
#This test is specific to the "Frequency of Building Types Sold by Year" plot in Visualizations notebook and slide 8 in presentation
chi2_contingency(df_house_all
    .pipe(remove_undeveloped_lots)
    .groupby("YrSold_YYYY").BldgType
    .value_counts(normalize = False)
    .unstack(level = 0)
    .reset_index()
    .filter([2019, 2021])
    .values
)[1]

0.19799938499757733